In [1]:
import os
import pickle
os.getcwd()
os.chdir("../../")

import numpy as np
from matplotlib import pyplot as plt

In [2]:
from src.gpdssm.models import StandardSingleModel

In [3]:
def init_model(fold, cycle=1, learning_rate=0.001):
    model = StandardSingleModel(data_name="train_2", data_fold=fold, dim_hidden=[2], num_rff=50, num_particle=100, 
                            display_step=2000, warm_start=0, learning_rate=learning_rate, num_train_cycle=cycle)
    return model

# def zero_round_learn(model_list):
#     for i in range(len(model_list)):
#         print("Class " + str(i) + " starts learning")
#         model_list[i].learn()

# def first_round_learn(model_list):
#     C = len(model_list)
#     new_model_list = [init_model(i, cycle_array[i-1]) for i in range(1, C+1)]
#     print("Class 0 starts learning")
#     new_model_list[0].initialize_structure_from_model(model_from_same_class = model_list[0])
#     new_model_list[0].learn()
    
#     for i in range(1, C):
#         print("Class " + str(i) + " starts learning")
#         new_model_list[i].initialize_structure_from_model(new_model_list[i-1], model_list[i])
#         new_model_list[i].learn()
        
#     return new_model_list

def first_round_learn(model_list, cycle_array, rate_array):
    C = len(model_list)
    new_model_list = [init_model(i, cycle_array[i-1], rate_array[i-1]) for i in range(1, C+1)]

    print("Class 0 starts learning")
    new_model_list[0].learn()
    
    for i in range(1, C):
        print("Class " + str(i) + " starts learning")
        new_model_list[i].initialize_structure_from_model(new_model_list[i-1])
        new_model_list[i].learn()
        
    return new_model_list
        
# def next_round_learn(model_list, cycle_array, rate_array, stop, flag):
#     C = len(model_list)
#     new_model_list = [init_model(i, cycle_array[i-1], rate_array[i-1]) for i in range(1, C+1)]
    
#     # if not stop[0]:
#     #     print("Class 0 starts learning")
#     #     new_model_list[0].initialize_structure_from_model(model_list[flag], model_list[0])
#     #     new_model_list[0].relearn()
#     #     flag = 0
#     # else:
#     #     new_model_list[0] = model_list[0]
    
#     for i in range(C):
#         if not stop[i]:
#             print("Class " + str(i) + " starts learning")
#             if flag < i:
#                 new_model_list[i].initialize_structure_from_model(new_model_list[flag], model_list[i])
#             else:
#                 new_model_list[i].initialize_structure_from_model(model_list[flag], model_list[i])
#             new_model_list[i].relearn()
#             flag = i
#         else:
#             new_model_list[i] = model_list[i]
        
            
#     return new_model_list, flag

def next_round_learn(model_list, cycle_array, rate_array, stop, flag):
    C = len(model_list)
    new_model_list = [init_model(i, cycle_array[i-1], rate_array[i-1]) for i in range(1, C+1)]
    
    # if not stop[0]:
    print("Class 0 starts learning")
    new_model_list[0].initialize_structure_from_model(model_list[-1], model_list[0])
    new_model_list[0].relearn()
    #     flag = 0
    # else:
    #     new_model_list[0] = model_list[0]
    
    for i in range(1, C):
        # if not stop[i]:
        #     print("Class " + str(i) + " starts learning")
        #     if flag < i:
        #         new_model_list[i].initialize_structure_from_model(new_model_list[flag], model_list[i])
        #     else:
        #         new_model_list[i].initialize_structure_from_model(model_list[flag], model_list[i])
        #     new_model_list[i].relearn()
        #     flag = i
        # else:
        #     new_model_list[i] = model_list[i]
        print("Class " + str(i) + " starts learning")
        new_model_list[i].initialize_structure_from_model(new_model_list[i-1], model_list[i])
        new_model_list[i].relearn()
            
    return new_model_list

def get_mse_list(model_list):
    return [model_list[i].get_mse()[-1] for i in range(len(model_list))]

def check_stop(mse_list, model_list):
    new_mse_list = get_mse_list(model_list)
    ratio = np.array(new_mse_list) / np.array(mse_list)
    return new_mse_list, (ratio > 0.98) & (ratio < 1.02)

In [4]:
C = 3
cycle = [1, 1, 1]
rate = [0.005, 0.001, 0.001]
stop = [0, 0, 0]
flag = C - 1

In [5]:
model_list = [init_model(i, cycle[i-1], rate[i-1]) for i in range(1, C+1)]

model_list = first_round_learn(model_list, cycle, rate)
# model_list[2].learn()

Class 0 starts learning
>>> train cycle 1
>>> cycle=1, t=1, mse=1.8214, mnll=6.2214, time=0.0 minutes
>>> cycle=1, t=2000, mse=0.4431, mnll=-11.3021, time=0.11 minutes
>>> cycle=1, t=4000, mse=0.2303, mnll=-16.1969, time=0.22 minutes
>>> cycle=1, t=6000, mse=0.1572, mnll=-19.051, time=0.34 minutes
>>> cycle=1, t=8000, mse=0.1203, mnll=-21.0473, time=0.46 minutes
>>> cycle=1, t=10000, mse=0.0981, mnll=-22.5689, time=0.59 minutes
Class 1 starts learning
>>> train cycle 1
>>> cycle=1, t=1, mse=4.6822, mnll=-225221.8585, time=0.0 minutes
>>> cycle=1, t=2000, mse=1.5701, mnll=-122.2715, time=0.11 minutes
>>> cycle=1, t=4000, mse=1.4466, mnll=-71.7033, time=0.22 minutes
>>> cycle=1, t=6000, mse=1.4163, mnll=-55.0026, time=0.34 minutes
>>> cycle=1, t=8000, mse=1.3913, mnll=-46.7095, time=0.46 minutes
>>> cycle=1, t=10000, mse=1.3745, mnll=-41.9855, time=0.58 minutes
Class 2 starts learning
>>> train cycle 1
>>> cycle=1, t=1, mse=3.8018, mnll=-419500.3422, time=0.0 minutes
>>> cycle=1, t=2000,

In [6]:
# for i in range(1):
#     model_list = next_round_learn(model_list, cycle, rate, stop, flag)
#     # mse_list, stop = check_stop(mse_list, model_list)

In [7]:
cwd = os.getcwd()
model_path = os.path.join(cwd, "models", "train_2.pickle")
with open(model_path, 'wb') as f:
    pickle.dump(model_list, f)